![](./figures/Logo.PNG)

## In this part of the tutorial, you will
* use a genetic algorithm to optimize model outputs globally
* critically assess the optimized parameters and objective function results
* implement and use your own objective function(s) 

- - -

# 2 d - Optimization

- - -

## 1 About optimization

In the previous tutorials, we have relied on our own capacity to tune the input parameters of our model. In this tutorial, we will use an algorithm to do this job for us by minimizing an objective function we define. Such algorithms run a model multiple times and determine the optimal combination of input parameters by computing the objective function for each model output. 

---

### <div class="blue"><span style="color:blue">Exercise section</span></div>
### Exercise 1 - Optimization

(a) What problems may occur when relying purely on an algorithm to optimize a model?

* Answer

---

## 2 Using global optimization

**Import packages**

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import random
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
import sys
sys.path.append('src/')
import HyMod
from ipywidgets import interact, Dropdown

**Defining functions**

In [2]:
def bias(obs, sim):
    """
    Calculates bias
    obs :  The observed time series
    sim :  The simulated time series
    @returns : The bias
    """
    return np.mean(obs - sim)


def rmse(obs, sim):
    """
    Calculates root mean squared error
    obs :  The observed time series
    sim :  The simulated time series
    @returns : The RMSE
    """
    return np.sqrt(np.mean((obs - sim)**2))

def one_minus_nse(obs, sim):
    """
    Calculates Nash Sutcliffe efficiency
    obs :  The observed time series
    sim :  The simulated time series
    @returns : 1-NSE(obs, sim)
    """
    return 1-(1 - (np.sum((obs - sim)**2)) / (np.sum((obs - np.mean(obs))**2)))

def hymod_func(param, precip, pet, runoff_obs, objective_fun, n_days):
    #print(f"Parameters: Sm = {param[0]}, beta = {param[1]}, alfa = {param[2]}, Rs = {param[3]}, Rf = {param[4]}")
    param = np.array([param[0], param[1], param[2], 1/param[3], 1/param[4]])  # divide 1 by Rs and Rf for this HyMod implementation
    runoff_sim, states, fluxes = HyMod.hymod_sim(param, precip, pet)
    res = objective_fun(runoff_obs[n_days:], runoff_sim[n_days:])
    #print(f"Result of objective function: {res}")
    return res

**Create and display interactive menus for selecting catchment**

In [3]:
catchment_names = ["Siletz River, OR, USA", "Medina River, TX, USA", "Trout River, BC, Canada"]
dropdown = Dropdown(
    options=catchment_names,
    value=catchment_names[0],
    description='Catchment:',
    disabled=False)

display(dropdown)

Dropdown(description='Catchment:', options=('Siletz River, OR, USA', 'Medina River, TX, USA', 'Trout River, BC…

**Read catchment data and prepare model input**

In [4]:
# Read catchment data
catchment_name = dropdown.value
# Read catchment data
file_dic = {catchment_names[0]: "camels_14305500", catchment_names[1]: "camels_08178880", catchment_names[2]: "hysets_10BE007"}
df_obs = pd.read_csv(f"data/{file_dic[catchment_name]}.csv")
# Make sure the date is interpreted as a datetime object -> makes temporal operations easier
df_obs.date = pd.to_datetime(df_obs['date'], format='%Y-%m-%d')
# Select time frame
start_date = '2002-10-01'
end_date = '2004-09-30'
spinup_days = 365

# Index frame by date
df_obs.set_index('date', inplace=True)
# Select time frame
df_obs = df_obs[start_date:end_date]
# Reformat the date for plotting
df_obs["date"] = df_obs.index.map(lambda s: s.strftime('%b-%d-%y'))
# Reindex
df_obs = df_obs.reset_index(drop=True)
# Select snow, precip, PET, streamflow and T
df_obs = df_obs[["snow_depth_water_equivalent_mean", "total_precipitation_sum","potential_evaporation_sum","streamflow", "temperature_2m_mean", "date"]]
# Rename variables
df_obs.columns = ["Snow [mm/day]", "P [mm/day]", "PET [mm/day]", "Q [mm/day]", "T [C]", "Date"]

# Prepare the data intput for both models
P = df_obs["P [mm/day]"].to_numpy()
evap = df_obs["PET [mm/day]"].to_numpy()
temp = df_obs["T [C]"].to_numpy()
q_obs = df_obs["Q [mm/day]"].to_numpy()

**Using global optmization**

The code below runs as sequence of three optimization trials with a low iteration count to show potential differences between the trials.

In [5]:
max_iter = 5  # set maximum number of iterations performed by genetic algorithm
bounds = [(0,400), (0, 2), (0, 1), (2, 200), (1, 7)]  # set bounds for parameters
objective_functions = {"Bias":bias, "RMSE": rmse, "1-NSE": one_minus_nse} 
OF = "Bias"  # alternatively, you can choose "Bias", "1-NSE" or "RMSE"

print(f"Optimizing HyMOD for {catchment_name} (iterating up to {max_iter} times)")
for cal_round in range(1, 4):
    # generate inital parameters randomly
    x0 = []
    for i in bounds:
        low, high = i
        x0.append(round(random.uniform(low,high), 2))
    print(f"  Trial {cal_round}")
    print(f"  Random intial parameters: Sm = {x0[0]}, beta = {x0[1]}, alfa = {x0[2]}, Rs = {x0[3]}, Rf = {x0[4]}")

    # run the genetic algorithm to optimize the parameters
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html
    # "bounds" are the parameter ranges
    # "args" are the arguements for the evaluated function (hymod_func)
    # "x0" is the initial guess or starting point for the optimization (the random parameters generated above)
    # "max_iter" is the maximum number of iterations we allow the opitmization algorithm to do
    hymod_calib = differential_evolution(hymod_func, bounds=bounds, args=(P, evap, q_obs, objective_functions[OF], spinup_days), x0=x0, maxiter=max_iter)
    parameters = hymod_calib.x  # extract parameters resulting from calibration
    
    # divide 1 by Rs and Rf for this HyMod implementation
    parameters = np.array([parameters[0], parameters[1], parameters[2], 1/parameters[3], 1/parameters[4]])
    parameters = np.round(parameters, decimals=3)  # round parameters to 3 decimals
    q_sim, states, fluxes = HyMod.hymod_sim(parameters, P, evap)  # run HyMod
    # calculate the result of the objective function for the optimized parameters
    obj_fun_result = objective_functions[OF](q_obs[spinup_days:], q_sim[spinup_days:])  
    
    # print calibration results
    parameters = np.array([parameters[0], parameters[1], parameters[2], 1/parameters[3], 1/parameters[4]])
    parameters = np.round(parameters, decimals=3)  # round parameters to 3 decimals
    print(f"  Optimization results:")
    print(f"    Best found parameter set: Sm = {parameters[0]}, beta = {parameters[1]}, alfa = {parameters[2]}, Rs = {parameters[3]}, Rf = {parameters[4]}")
    print(f"    (this is based on the best found result for {OF}: {round(obj_fun_result, 3)})\n")

Optimizing HyMOD for Siletz River, OR, USA (iterating up to 5 times)
  Trial 1
  Random intial parameters: Sm = 142.38, beta = 1.91, alfa = 0.21, Rs = 2.0, Rf = 3.0
  Optimization results:
    Best found parameter set: Sm = 0.0, beta = 1.101, alfa = 0.0, Rs = 2.0, Rf = 3.003
    (this is based on the best found result for Bias: 1.752)

  Trial 2
  Random intial parameters: Sm = 311.51, beta = 1.89, alfa = 0.31, Rs = 2.0, Rf = 3.0
  Optimization results:
    Best found parameter set: Sm = 0.0, beta = 0.864, alfa = 0.0, Rs = 2.0, Rf = 3.003
    (this is based on the best found result for Bias: 1.752)

  Trial 3
  Random intial parameters: Sm = 65.99, beta = 1.1, alfa = 0.91, Rs = 2.0, Rf = 3.0
  Optimization results:
    Best found parameter set: Sm = 0.0, beta = 0.361, alfa = 0.0, Rs = 2.0, Rf = 3.003
    (this is based on the best found result for Bias: 1.752)



---

### <div class="blue"><span style="color:blue">Exercise section</span></div>
### Exercise 2 - Using global optimization

(a) Run the optimization for different catchments and different objective functions (change the variable "OF"). Compare the result of the different catchments! Where does the model perform better or worse (remember, what climatic regions the individual catchments represent)?

* Answer

(b) Why do results of the optimized objective functions change between the trials?

* Answer

(c) Why do, for the same results of the optimized objective function, some input parameters vary significantly?

* Answer

(d) Implement two objective functions of your choice! (You may choose any of the ones we used in previous tutorials or implement new ones)

---

## Jupyter format settings

In [6]:
%%html 
<style>.blue {background-color: #8dc9fc;}</style>